In [1]:
from bs4 import BeautifulSoup

In [2]:
import json

In [19]:
import re
def clean_html(text):
    return re.sub(r'\s+', ' ', BeautifulSoup(text, 'html.parser').get_text().replace('Â\xa0', ' ')).strip()

In [130]:
def extract_type1_cards(html, card_name):
    res = []
    soup = BeautifulSoup(html, 'html.parser')
    
    content_body = soup.find('div', id='contentBody')
    c11text_div = content_body.find('div', class_='c11text') if content_body else None
    first_inner_div = c11text_div.find_all('div')[0] if c11text_div else None
    tables = first_inner_div.find_all('table') if first_inner_div else None

    if not tables:
        return None
    
    table1 = tables[0]

    caption_p = table1.find('caption').find('p') if table1.find('caption') else None
    text1 = caption_p.get_text(strip=True) if caption_p else ""
    text1 = text1.replace('Â\xa0', '').strip()

    details = []

    tbody = table1.find('tbody')
    if tbody:
        for tr in tbody.find_all('tr'):
            th = tr.find('th')
            td = tr.find('td')

            if not th or not td:
                continue

            th_p = th.find('p')
            text2 = th_p.get_text(strip=True) if th_p else ""
            text2 = text2.replace('Â\xa0', '').strip()

            text3 = " ".join(p.get_text(strip=True)+" " for p in td.find_all('p')) if td else ""
            text3 = text3.replace('Â\xa0', '').strip()

            details.append({
                "type": text2,
                "value": text3
            })
    res.append({"card_name": card_name, "section_type": text1, "details": details})

    table2 = tables[1]

    caption_p = table2.find('caption')
    text1 = clean_html(str(caption_p)) if caption_p else ""

    tbody2 = table2. find('tbody')
    text2, text3 = "", ""
    if tbody2:
        first_tr = tbody2.find('tr')
        if first_tr:
            th = first_tr.find('th')
            td = first_tr.find('td')
            
            text2 = clean_html(str(th)) if th else ""
            text3 = clean_html(str(td)) if td else ""

    table3 = tables[2]
    
    tbody3 = table3.find('tbody')
    text4 = ""
    nested_details = []
    if tbody3:
        rows = tbody3.find_all('tr')

        if rows:
            first_tr = rows[0]
            th = first_tr.find('th')
            text4 = clean_html(str(th)) if th else ""

        for i, tr in enumerate(rows[1:], start=1):
            th = tr.find('th')
            td = tr.find('td')

            text5_i = clean_html(str(th)) if th else ""
            text6_i = clean_html(str(td)) if td else ""
            text5_i = text5_i.replace('â€¢ ', '').strip()

            nested_details.append({"type": text5_i, "value": text6_i})
    
    table4 = tables[3]
    
    tbody4 = table4.find('tbody')
    text7, text8, text9 = "", "", ""
    if tbody4:
        rows = tbody4.find_all('tr')
        if rows:
            first_tr = rows[0]
            th = first_tr.find('th')
            text7 = clean_html(str(th)) if th else ""
        if len(rows) > 1:
            second_tr = rows[1]
            th = second_tr.find('th')
            td = second_tr.find('td')

            text8 = clean_html(str(th)) if th else ""
            text8 = text8.replace('â€¢ ', '').strip()
            text9 = clean_html(str(td)) if td else ""

    text10, text11 = "", ""

    h4 = table4.find_next('h4')
    if h4:
        text10 = clean_html(str(h4))

        p_tags = h4.find_all_next('p', limit=3)
        if len(p_tags) >= 3:
            text11 = clean_html(str(p_tags[0])) + " " + clean_html(str(p_tags[2]))

    res.append({"card_name": card_name, "section_type": text1, "details": [{"type": text2, "value": text3}, {"type": text4, "value": nested_details}, {"type": text7, "value": [{"type": text8, "value": text9}]}]})
    res.append({"card_name":card_name, "section_type": text10, "details": [{"question": text10, "answer": text11}]})

    ols = first_inner_div.find_all('ol')
    ol = None
    for o in reversed(ols):
        if not o.find_parent('li'):
            ol = o
            break
    h4 = ol.find_previous('h4')
    text1 = clean_html(str(h4)) if h4 else ""
    text1 = text1.split(":")[-1].strip()

    details = []
    for i, li in enumerate(ol.find_all('li'), start=1):
        if not li.find_parent('li'):
            h5 = li.find('h5')
            text3_parts = []
            for element in li.children:
                if element == h5:
                    continue
                if isinstance(element, str):
                    text3_parts.append(element.strip())
                elif hasattr(element, 'text'):
                    text3_parts.append(clean_html(str(element)))
            # p = li.find('p')

            text2 = clean_html(str(h5)) if h5 else ""
            # text3 = clean_html(str(p)) if p else ""
            text3 = ' '.join(text3_parts)
            details.append({"question": text2, "answer": text3})
    
    res.append({"card_name": card_name, "section_type": text1, "details": details})

    second_inner_div = c11text_div.find("div", {"role": "region"})
    if second_inner_div:
        h4 = second_inner_div.find('h4')
        text1 = clean_html(str(h4)).replace('<br>', ' ').strip()
        text1 = text1.split(":")[-1].strip()

        content_before_ol = ""
        for element in second_inner_div.find_all(True):
            if element == h4:
                continue
            if element.name == 'ol':
                break
            content_before_ol += clean_html(str(element)).strip() + " "
        
        text2 = content_before_ol.strip()

        ol = second_inner_div.find('ol')
        li_details = []

        for li in ol.find_all('li'):
            if not li.find_parent('li'):
                h6 = li.contents[0]
                text3 = clean_html(str(h6)) if h6 else ""
                text4_parts = []
                for element in li.children:
                    if element == h6:
                        continue
                    if isinstance(element, str):
                        text4_parts.append(element.strip())
                    elif hasattr(element, 'text'):
                        text4_parts.append(clean_html(str(element)))
                
                text4 = ' '.join(text4_parts)

                li_details.append({"question": text3, "answer": text4})
        
        res.append({"card_name": card_name, "section_type": text1, "details": [{"type": "Description", "value": text2}]})
        res.append({"card_name": card_name, "section_type": text1, "details": li_details})
    

    return res
    

In [131]:
html_content = open('htmls/activecashcard.html').read()
content = extract_type1_cards(html_content, "Active Cash Visa Card")
html_content2 = open('htmls/autographjourney.html').read()
content += extract_type1_cards(html_content2, "Autograph Journey Visa Card")
html_content3 = open('htmls/autographvisa.html').read()
content += extract_type1_cards(html_content3, "Autograph Visa Card")
html_content4 = open('htmls/reflect.html').read()
content += extract_type1_cards(html_content4, "Reflect Visa Card")
html_content5 = open('htmls/attune.html').read()
content += extract_type1_cards(html_content5, "Attune World Elite Mastercard")
html_content6 = open('htmls/onekey.html').read()
content += extract_type1_cards(html_content6, "One Key Mastercard")
html_content7 = open('htmls/onekeyplus.html').read()
content += extract_type1_cards(html_content7, "One Key Plus Mastercard")
json.dump(content, open('z.json', 'w'), indent=4, ensure_ascii=False)

In [124]:
def extract_type2_cards(html, card_name):
    res = []
    soup = BeautifulSoup(html, 'html.parser')

    content_body = soup.find('div', id='contentBody')
    c11text_div = content_body.find('div', class_='c11text') if content_body else None
    first_inner_div = c11text_div.find_all('div')[0] if c11text_div else None
    tables = first_inner_div.find_all('table') if first_inner_div else None

    if not tables:
        return None
    
    table1 = tables[0]

    caption_p = table1.find('caption').find('p') if table1.find('caption') else None
    text1 = caption_p.get_text(strip=True) if caption_p else ""
    text1 = text1.replace('Â\xa0', '').strip()

    details = []

    tbody = table1.find('tbody')
    if tbody:
        for tr in tbody.find_all('tr'):
            th = tr.find('th')
            td = tr.find('td')

            if not th or not td:
                continue

            th_p = th.find('p')
            text2 = th_p.get_text(strip=True) if th_p else ""
            text2 = text2.replace('Â\xa0', '').strip()

            text3 = " ".join(p.get_text(strip=True)+" " for p in td.find_all('p')) if td else ""
            text3 = text3.replace('Â\xa0', '').strip()

            details.append({
                "type": text2,
                "value": text3
            })
    res.append({"card_name": card_name, "section_type": text1, "details": details})

    table2 = tables[1]

    caption_p = table2.find('caption')
    text1 = clean_html(str(caption_p)) if caption_p else ""

    tbody2 = table2. find('tbody')
    text2, text3 = "", ""
    if tbody2:
        first_tr = tbody2.find('tr')
        if first_tr:
            th = first_tr.find('th')
            td = first_tr.find('td')
            
            text2 = clean_html(str(th)) if th else ""
            text3 = clean_html(str(td)) if td else ""

    table3 = tables[2]
    
    tbody3 = table3.find('tbody')
    text4 = ""
    nested_details = []
    if tbody3:
        rows = tbody3.find_all('tr')

        if rows:
            first_tr = rows[0]
            th = first_tr.find('th')
            text4 = clean_html(str(th)) if th else ""

        for i, tr in enumerate(rows[1:], start=1):
            th = tr.find('th')
            td = tr.find('td')

            text5_i = clean_html(str(th)) if th else ""
            text6_i = clean_html(str(td)) if td else ""
            text5_i = text5_i.replace('• ', '').strip()

            nested_details.append({"type": text5_i, "value": text6_i})
    
    table4 = tables[3]
    
    tbody4 = table4.find('tbody')
    text7, text8, text9 = "", "", ""
    if tbody4:
        rows = tbody4.find_all('tr')
        if rows:
            first_tr = rows[0]
            th = first_tr.find('th')
            text7 = clean_html(str(th)) if th else ""
        if len(rows) > 1:
            second_tr = rows[1]
            th = second_tr.find('th')
            td = second_tr.find('td')

            text8 = clean_html(str(th)) if th else ""
            text8 = text8.replace('• ', '').strip()
            text9 = clean_html(str(td)) if td else ""

    res.append({"card_name": card_name, "section_type": text1, "details": [{"type": text2, "value": text3}, {"type": text4, "details": nested_details}, {"type": text7, "value": [{"type": text8, "value": text9}]}]})

    h4s = first_inner_div.find_all('h4')

    for h4 in h4s:
        text1 = clean_html(str(h4))
        
        text2_parts = []
        current_element = h4.find_next_sibling()

        while current_element and current_element.name != 'h4':
            if isinstance(current_element, str):
                text2_parts.append(current_element.strip())
            elif hasattr(current_element, 'text'):
                text2_parts.append(clean_html(str(current_element)))
            current_element = current_element.find_next_sibling()
        
        text2 = ' '.join(text2_parts).strip()
        res.append({"card_name": card_name, "section_type": text1, "details": text2})

    hr = c11text_div.find('hr')
    second_inner_div = hr.find_next('div')
    # for div in reversed(divs):
    #     if not div.find_parent('li') and div != first_inner_div:
    #         second_inner_div = div
    #         break
    
    h2 = second_inner_div.find('h2')
    print(second_inner_div)
    text1 = clean_html(str(h2)) if h2 else ""

    ol = second_inner_div.find('ol')
    li_list = []
    for li in ol.find_all('li'):
        if li.find_parent('li'):
            continue
        li_list.append(li)

    details = []
    
    for li in li_list:
        p = li.find('p')
        text2 = clean_html(str(p)) if p else ""

        sub_list = li.find(['ul', 'ol'])
        sub_details = []

        if sub_list:
            sub_li_list = sub_list.find_all('li')
            for sub_li in sub_li_list:
                if not sub_li.find_parent('ul'):
                    sub_text = clean_html(str(sub_li))
                    sub_details.append({"type": text2, "value": sub_text})
        details.extend(sub_details)
    
    res.append({"card_name": card_name, "section_type": text1, "details": details})

    return res

In [125]:
html_content8 = open('htmls/bilt.html', encoding='utf-8').read()
content2 = extract_type2_cards(html_content8, "Bilt World Elite Mastercard")
json.dump(content2, open('x.json', 'w'), indent=4, ensure_ascii=False)

<div style="color:black;font-family:'helvetica';font-size:10pt;margin-top:8pt;text-align:justify"><h2 style="font-size:12pt;margin-bottom:0pt;text-align:center"><a id="rewardssummary"></a><div style="text-align:center"><strong>Bilt World Elite Mastercard<sup>®</sup> Credit Card Rewards<br/>Program Agreement (the "Card Rewards Program") Terms and Conditions ("Terms")</strong></div></h2><table role="presentation"><tbody><tr><th>Section 1:</th><td>Definitions</td></tr><tr><th>Section 2:</th><td>Agreement</td></tr><tr><th>Section 3:</th><td>Earning points with your Bilt Mastercard</td></tr><tr><th>Section 4:</th><td>Points Redemption</td></tr><tr><th>Section 5:</th><td>General Terms</td></tr><tr><th>Section 6:</th><td>Governing Law</td></tr><tr><th>Section 7:</th><td>Arbitration Disclosure</td></tr></tbody></table><ol style="margin-bottom:3pt;margin-top:3pt"><li style="margin-top:3pt"><p><strong>Definitions.</strong> The terms below have the following meanings:</p><ul style="list-style-typ

In [101]:
content2

[{'card_name': 'Bilt World Elite Mastercard',
  'section_type': '',
  'details': [{'type': 'Annual Percentage Rate (APR) for Purchases',
    'value': '20.49%, 23.49%,or28.49%, based on your creditworthiness.  This APR will vary with the market based on the U.S. Prime Rate.'},
   {'type': 'Annual Percentage Rate (APR) for Balance Transfers',
    'value': '20.49%, 23.49%,or28.49%, based on your creditworthiness. Balance Transfers are subject to eligibility.  This APR will vary with the market based on the U.S. Prime Rate.'},
   {'type': 'APR for Cash Advances',
    'value': '28.24%  This APR will vary with the market based on the U.S. Prime Rate.'},
   {'type': 'How to Avoid Paying Interest on Purchases',
    'value': 'Your due date is at least 25 days after the close of each billing period. We will not charge you interest on purchases if you pay your entire balance by the due date each month.'},
   {'type': 'Minimum Interest Charge',
    'value': 'If you are charged interest, the charge